In [ ]:
###### In this notebook we're going to compute the overlap metrics between T5 and the baselines ######

In [ ]:
def computePerfectPrediction(predictions_file, target_file):

    perfect_pred = {}


    target = open(target_file,'r')
    target_list =  target.readlines()

    for i in range(1,16):

        perfect_pred[i] = 0

        with open(predictions_file) as fread:

            data = fread.readlines()
            
            perfect_file = open('perfect_{}.txt'.format(i),'a+')
            for (ref,pred) in zip(target_list,data):

                pred = pred.strip().lower().split()
                pred = pred[0:i]

                ref = ref.strip().lower().split()
                ref = ref[0:i]

                if len(ref)>=i and len(pred)>=i:
                    if ''.join(pred) == ''.join(ref):
                        perfect_pred[i]+=1
                        perfect_file.write(' '.join(pred)+'\n')
            perfect_file.close()

            len_tokens_subset = len([item.strip() for item in target_list if len(item.split())>=i ])
            perfect_pred[i] = (perfect_pred[i], len_tokens_subset)
            
    return perfect_pred

path_target_file = ''
path_pred_file = ''
computePerfectPrediction(path_target_file, path_pred_file)

In [ ]:
### LOAD T5 AND 5-GRAM PREDICTIONS ####

perfect_pred_t5 = []
perfect_pred_5gram = []

for i in range(1,16):
    
    with open('T5_INSIDE/perfect_{}.txt'.format(i)) as fread:
        for item in fread.readlines():
            item = item.strip()
            perfect_pred_t5.append(item)
    
    with open('5GRAM_INSIDE/perfect_{}.txt'.format(i)) as fread:
        for item in fread.readlines():
            item = item.strip()
            perfect_pred_5gram.append(item)
        
        
perfect_pred_t5_set = [''.join(perfect.split()) for perfect in perfect_pred_t5 ]
perfect_pred_5gram_set = [''.join(perfect.split()) for perfect in perfect_pred_5gram ]

In [ ]:
###### OVERLAP #####

set_t5 = set(perfect_pred_t5_set)
set_baseline = set(perfect_pred_5gram_set)

#### SHARED #####
intersection = set_t5.intersection(set_baseline)
union = set_t5.union(set_baseline)
shared = (len(intersection)/len(union) ) * 100

print('Shared: ',shared)
# print(len(intersection))

#### ONLY T5 #####
difference = set_t5.difference(set_baseline)
union = set_t5.union(set_baseline)
t5_only = (len(difference)/len(union) ) * 100

# print(difference)

print('Only T5: ',t5_only)
# print(len(intersection))

#### ONLY BASELINE #####
difference = set_baseline.difference(set_t5)
union = set_t5.union(set_baseline)
baseline_only = (len(difference)/len(union) ) * 100

# print(difference)

print('Only BASELINE: ',baseline_only)

assert(t5_only + shared + baseline_only > 99.9)